In [1]:
from ib_insync import *
import pandas as pd
from datetime import datetime, timedelta
import requests
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

In [2]:
util.startLoop()

In [3]:
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

<IB connected to 127.0.0.1:7497 clientId=1>

In [4]:
# Definir el contrato para la acción de Palantir Technologies Inc. (PLTR)
contract = Stock('PLTR', 'SMART', 'USD')


In [5]:
# Establecer el rango de tiempo
end_time = datetime.now()
start_time = end_time - timedelta(days=360)


In [6]:
# Obtener datos históricos
bars = ib.reqHistoricalData(
    contract,
    endDateTime=end_time,
    durationStr='360 D',
    barSizeSetting='1 hour',
    whatToShow='TRADES',
    useRTH=False
)

In [7]:
# Convertir los datos a DataFrame
df = util.df(bars)

In [8]:
# Obtener noticias relevantes sobre Palantir Technologies Inc. (PLTR)
API_KEY = 'b35c56d955ee45178c703f7f79c1dfca'
news_url = f'https://newsapi.org/v2/everything?q=Palantir%20OR%20NYSE%20PLTR&apiKey={API_KEY}'
response = requests.get(news_url)
news_data = response.json()

In [9]:
titles = [article['title'] for article in news_data['articles']]
print(f"Titulares de noticias relevantes:\n {titles}")


Titulares de noticias relevantes:
 ['Palantir (PLTR) Shares Skyrocket, What You Need To Know', 'Palantir Sparks AI Frenzy with Teaser--Big Reveal on March 13', 'Palantir Technologies (PLTR) Stock Bulls Get Set to Punish Panicked Sellers', "Palantir won't be hurt by Pentagon cuts, analyst says — and the stock pops 5%", 'Palantir stock slips more than 8% as the bleeding gets worse', 'Nvidia, Tesla, Oracle, Palantir, Coinbase, Doordash: Stocks to watch today', 'Intel, Nvidia, Tesla, Palantir, IRobot: Stocks to watch today', 'Will Palantir Stock Fall To $40?', 'Walmart, Alibaba, Palantir, Meta, Hasbro, Block, Cruise Lines: Stocks to watch today', 'Shares in “Unique” Palantir ($PLTR) Rise On Hopes it Can Deflect Trump Cost Crunch', 'Morgan Stanley cautions that the S&P 500 could suffer a 5% drop', "Nvidia and Big Tech stocks can't escape the market's growing anxiety", '1 Artificial Intelligence (AI) Stock to Buy Hand Over Fist Before It Surges by 60%, According to 1 Wall Street Analyst', 'P

In [10]:
# Análisis de sentimiento de noticias
positive_keywords = ['rise', 'growth', 'record', 'profit']
negative_keywords = ['fall', 'loss', 'decline', 'drop']


In [11]:
positive_news = sum(1 for title in titles if any(word in title.lower() for word in positive_keywords))
negative_news = sum(1 for title in titles if any(word in title.lower() for word in negative_keywords))
impact_factor = 1 + (positive_news - negative_news) * 0.001

In [12]:
# Seleccionar datos y normalizar
data = df[['open', 'high', 'low', 'close']].values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)
scaled_data_adjusted = scaled_data * impact_factor


In [13]:
# Identificar soportes y resistencias
def find_support_resistance(data):
    levels = []
    for i in range(2, len(data) - 2):
        if data[i] < data[i - 1] and data[i] < data[i + 1]:
            levels.append((i, data[i]))  # Soporte
        elif data[i] > data[i - 1] and data[i] > data[i + 1]:
            levels.append((i, data[i]))  # Resistencia
    return levels

support_resistance_levels = find_support_resistance(df['close'].values)
print(f"Niveles de soporte y resistencia detectados: {support_resistance_levels}")


Niveles de soporte y resistencia detectados: [(4, np.float64(17.23)), (5, np.float64(16.89)), (6, np.float64(17.33)), (7, np.float64(17.31)), (10, np.float64(17.435)), (13, np.float64(17.3)), (14, np.float64(17.3491)), (18, np.float64(17.2)), (19, np.float64(17.23)), (20, np.float64(17.0)), (24, np.float64(17.925)), (25, np.float64(17.665)), (27, np.float64(17.84)), (28, np.float64(17.81)), (29, np.float64(17.83)), (34, np.float64(17.64)), (35, np.float64(17.71)), (36, np.float64(17.64)), (37, np.float64(17.725)), (38, np.float64(17.455)), (39, np.float64(17.56)), (40, np.float64(17.361)), (41, np.float64(17.445)), (43, np.float64(17.2)), (44, np.float64(17.3199)), (48, np.float64(17.1)), (52, np.float64(17.36)), (57, np.float64(17.24)), (63, np.float64(16.89)), (64, np.float64(17.02)), (66, np.float64(16.91)), (67, np.float64(17.01)), (70, np.float64(15.825)), (73, np.float64(16.335)), (76, np.float64(16.02)), (77, np.float64(16.05)), (78, np.float64(16.0394)), (79, np.float64(16.09))

In [14]:
# Crear secuencias de datos para LSTM
sequence_length = 60
x_train, y_train = [], []
for i in range(sequence_length, len(scaled_data_adjusted)):
    x_train.append(scaled_data_adjusted[i-sequence_length:i])
    y_train.append(scaled_data_adjusted[i])

x_train, y_train = np.array(x_train), np.array(y_train)

In [15]:
# Definir modelo LSTM
model = Sequential()
model.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=4))

# Compilar y entrenar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - loss: 0.0166
Epoch 2/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 0.0021
Epoch 3/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 0.0018
Epoch 4/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 0.0014
Epoch 5/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 0.0011
Epoch 6/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 0.0010
Epoch 7/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 8.8633e-04
Epoch 8/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 8.0611e-04
Epoch 9/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - loss: 8.1272e-04
Epoch 10/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - loss: 7.5916e-04
Epoch 11/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - loss: 7.9451e-04
Epoch 12/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - loss: 7.4158e-04
Epoch 13/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - loss: 5.9223e-04
Epoch 14/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - loss: 6.5706e-04
Epoch 15/20

In [18]:
# Definir la hora de inicio de la predicción
start_prediction_time = datetime.now().replace(hour=15, minute=0, second=0, microsecond=0)


In [19]:
# Predicciones para una semana
recent_data = scaled_data_adjusted[-sequence_length:]
predictions = []
for _ in range(7 * 24):  # Predicción para 7 días en intervalos de 1 hora
    x_input = np.array([recent_data])
    predicted_price = model.predict(x_input)
    predictions.append(predicted_price[0])
    recent_data = np.append(recent_data[1:], [predicted_price[0]], axis=0)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━

In [20]:
# Desnormalizar predicciones
predictions = scaler.inverse_transform(predictions)
prediction_times = [start_prediction_time + timedelta(hours=i) for i in range(1, 7 * 24 + 1)]
predicted_df = pd.DataFrame(predictions, columns=['open', 'high', 'low', 'close'], index=prediction_times)
print(predicted_df)


                          open       high        low      close
2025-03-20 16:00:00  86.094128  86.336692  85.535878  85.630962
2025-03-20 17:00:00  85.860751  86.108319  85.310620  85.396995
2025-03-20 18:00:00  85.571570  85.822430  85.027908  85.106143
2025-03-20 19:00:00  85.256017  85.509292  84.716781  84.789151
2025-03-20 20:00:00  84.928636  85.183857  84.392524  84.461339
...                        ...        ...        ...        ...
2025-03-27 11:00:00  62.223251  62.504508  62.040571  62.081182
2025-03-27 12:00:00  62.172329  62.453580  61.990250  62.030867
2025-03-27 13:00:00  62.121919  62.403157  61.940435  61.981042
2025-03-27 14:00:00  62.072003  62.353239  61.891123  61.931723
2025-03-27 15:00:00  62.022592  62.303823  61.842296  61.882890

[168 rows x 4 columns]
